# Cryptocurrency Prediction: Time series forcasting of Bitcoin price

### By Ubong Ben

![](https://images.unsplash.com/photo-1621501011941-c8ee93618c9a?ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8fA%3D%3D&auto=format&fit=crop&w=872&q=80)

<iframe width="1024" height="576" src="https://www.youtube.com/embed/p-QY7JNGD60" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

# 2. Importing Library

In [1]:
#!pip install tensorflow

In [2]:
'''# First we will import the necessary Library

import os
import pandas as pd
import numpy as np
import math
import datetime as dt
import matplotlib.pyplot as plt

# For Evalution we will use these library

from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import LSTM

import matplotlib.pyplot as plt
from itertools import cycle
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots'''

'# First we will import the necessary Library\n\nimport os\nimport pandas as pd\nimport numpy as np\nimport math\nimport datetime as dt\nimport matplotlib.pyplot as plt\n\n# For Evalution we will use these library\n\nfrom sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score\nfrom sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score\nfrom sklearn.preprocessing import MinMaxScaler\n\nimport tensorflow as tf\nfrom tensorflow.keras.models import Sequential\nfrom tensorflow.keras.layers import Dense, Dropout\nfrom tensorflow.keras.layers import LSTM\n\nimport matplotlib.pyplot as plt\nfrom itertools import cycle\nimport plotly.graph_objects as go\nimport plotly.express as px\nfrom plotly.subplots import make_subplots'

# Preliminary Data Wrangling

#  [We can use this link to download bitcoin dataset from yahoo finance](https://finance.yahoo.com/quote/BTC-USD/history?p=BTC-USD)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

file_path = "/content/drive/My Drive/Colab Notebooks/Bitcoin Predictor/BTC-USD.csv"

# Load our dataset
maindf=pd.read_csv(file_path)
maindf.head()

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
#get total number of days in the dataset for analysis
print(maindf.shape[0])


Okay, Lets check the bottom to see if the dataset is consistent

In [ ]:
maindf.tail()

Not looking bad!!!!!!


In [ ]:
maindf.info()

In [ ]:
maindf.describe()

# Exploratory Data Analysis

We know the number of days that we are working with. However, it would be helpful if we knew the actual dates itself. What is our starting and ending date?

In [ ]:

print('The dataset starts at:',maindf.iloc[0][0])
print('The dataset ends at: ',maindf.iloc[-1][0])


### ANNUAL EXPLORATORY ANALYSIS OF OUR DATASET

![](https://www.questionpro.com/blog/wp-content/uploads/2022/05/exploratory-data-analysis.jpg)

#### YEAR 2014

In [ ]:
#How does this year look like as a whole?
maindf['Date'] = pd.to_datetime(maindf['Date'], format='%Y-%m-%d')
year_2014 = maindf.loc[(maindf['Date'] >= '2014-09-17') & (maindf['Date'] <= '2014-12-31')]
year_2014 .drop(year_2014 [['Adj Close','Volume']],axis=1)

In [ ]:
#how is the price distributed on the basis of month?
monthly= year_2014.groupby(year_2014['Date'].dt.strftime('%B'))[['Open','Close']].mean()
month_name = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August',
             'September', 'October', 'November', 'December']
monthly = monthly.reindex(month_name, axis=0)
monthly

In [ ]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x=monthly .index,
    y=monthly ['Open'],
    name='Stock Open Price'
))
fig.add_trace(go.Bar(
    x=monthly .index,
    y=monthly ['Close'],
    name='Stock Close Price',
))

fig.update_layout(barmode='group', xaxis_tickangle=-45,
                  title='2014 Monthly comparative analysis')
fig.show()

**What about the opening and closing prices for months in the year**

In [ ]:
year_2014.groupby(year_2014['Date'].dt.strftime('%B'))['Low'].min()
monthvise_high = year_2014.groupby(maindf['Date'].dt.strftime('%B'))['High'].max()
monthvise_high = monthvise_high.reindex(month_name, axis=0)

monthvise_low = year_2014.groupby(year_2014['Date'].dt.strftime('%B'))['Low'].min()
monthvise_low = monthvise_low.reindex(month_name, axis=0)

fig = go.Figure()
fig.add_trace(go.Bar(
    x=monthvise_high.index,
    y=monthvise_high,
    name='Stock high Price',
))
fig.add_trace(go.Bar(
    x=monthvise_low.index,
    y=monthvise_low,
    name='Stock low Price',
))

fig.update_layout(barmode='group',
                  title='High and Low Price 2014 (Monthly)')
fig.show()

**Now lets get a more combined signature**

In [ ]:
names = cycle(['Stock Open Price','Stock Close Price','Stock High Price','Stock Low Price'])

fig = px.line(year_2014, x=year_2014.Date, y=[year_2014['Open'], year_2014['Close'],
                                          year_2014['High'], year_2014['Low']],
             labels={'Date': 'Date','value':'Stock value'})
fig.update_layout(title_text='Stock analysis chart', font_size=15, font_color='black',legend_title_text='Stock Parameters')
fig.for_each_trace(lambda t:  t.update(name = next(names)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

fig.show()

#### YEAR 2015

In [ ]:
#How does this year look like as a whole?
maindf['Date'] = pd.to_datetime(maindf['Date'], format='%Y-%m-%d')
year_2015 = maindf.loc[(maindf['Date'] >= '2015-01-01') & (maindf['Date'] <= '2015-12-31')]
year_2015.drop(year_2015[['Adj Close','Volume']],axis=1)

In [ ]:
#how is the price distributed on the basis of month?
monthly= year_2015.groupby(year_2015['Date'].dt.strftime('%B'))[['Open','Close']].mean()
month_name = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August',
             'September', 'October', 'November', 'December']
monthly = monthly.reindex(month_name, axis=0)
monthly

In [ ]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x=monthly .index,
    y=monthly ['Open'],
    name='Stock Open Price'
))
fig.add_trace(go.Bar(
    x=monthly .index,
    y=monthly ['Close'],
    name='Stock Close Price',
))

fig.update_layout(barmode='group', xaxis_tickangle=-45,
                  title='2015 Monthly comparative analysis')
fig.show()

**What about the opening and closing prices for months in the year**

In [ ]:
year_2015.groupby(year_2015['Date'].dt.strftime('%B'))['Low'].min()
monthvise_high = year_2015.groupby(maindf['Date'].dt.strftime('%B'))['High'].max()
monthvise_high = monthvise_high.reindex(month_name, axis=0)

monthvise_low = year_2015.groupby(year_2015['Date'].dt.strftime('%B'))['Low'].min()
monthvise_low = monthvise_low.reindex(month_name, axis=0)

fig = go.Figure()
fig.add_trace(go.Bar(
    x=monthvise_high.index,
    y=monthvise_high,
    name='Stock high Price',
))
fig.add_trace(go.Bar(
    x=monthvise_low.index,
    y=monthvise_low,
    name='Stock low Price',
))

fig.update_layout(barmode='group',
                  title='High and Low Price 2015 (Monthly)')
fig.show()

**Now lets get a more combined signature**

In [ ]:
names = cycle(['Stock Open Price','Stock Close Price','Stock High Price','Stock Low Price'])

fig = px.line(year_2015, x=year_2015.Date, y=[year_2015['Open'], year_2015['Close'],
                                          year_2015['High'], year_2015['Low']],
             labels={'Date': 'Date','value':'Stock value'})
fig.update_layout(title_text='Stock analysis chart 2017', font_size=15, font_color='black',legend_title_text='Stock Parameters')
fig.for_each_trace(lambda t:  t.update(name = next(names)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

fig.show()

#### YEAR 2016

In [ ]:
#How does this year look like as a whole?
maindf['Date'] = pd.to_datetime(maindf['Date'], format='%Y-%m-%d')
year_2016 = maindf.loc[(maindf['Date'] >= '2016-01-01') & (maindf['Date'] <= '2016-12-31')]
year_2016.drop(year_2015[['Adj Close','Volume']],axis=1)

In [ ]:
#how is the price distributed on the basis of month?
monthly= year_2016.groupby(year_2016['Date'].dt.strftime('%B'))[['Open','Close']].mean()
month_name = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August',
             'September', 'October', 'November', 'December']
monthly = monthly.reindex(month_name, axis=0)
monthly

In [ ]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x=monthly .index,
    y=monthly ['Open'],
    name='Stock Open Price'
))
fig.add_trace(go.Bar(
    x=monthly .index,
    y=monthly ['Close'],
    name='Stock Close Price',
))

fig.update_layout(barmode='group', xaxis_tickangle=-45,
                  title='2016 Monthly comparative analysis')
fig.show()

**What about the opening and closing prices for months in the year**

In [ ]:
year_2016.groupby(year_2016['Date'].dt.strftime('%B'))['Low'].min()
monthvise_high = year_2016.groupby(maindf['Date'].dt.strftime('%B'))['High'].max()
monthvise_high = monthvise_high.reindex(month_name, axis=0)

monthvise_low = year_2016.groupby(year_2016['Date'].dt.strftime('%B'))['Low'].min()
monthvise_low = monthvise_low.reindex(month_name, axis=0)

fig = go.Figure()
fig.add_trace(go.Bar(
    x=monthvise_high.index,
    y=monthvise_high,
    name='Stock high Price',
))
fig.add_trace(go.Bar(
    x=monthvise_low.index,
    y=monthvise_low,
    name='Stock low Price',
))

fig.update_layout(barmode='group',
                  title='High and Low Price 2016 (Monthly)')
fig.show()

**Now lets get a more combined signature**

In [ ]:
names = cycle(['Stock Open Price','Stock Close Price','Stock High Price','Stock Low Price'])

fig = px.line(year_2016, x=year_2016.Date, y=[year_2016['Open'], year_2016['Close'],
                                          year_2016['High'], year_2016['Low']],
             labels={'Date': 'Date','value':'Stock value'})
fig.update_layout(title_text='Stock analysis chart', font_size=15, font_color='black',legend_title_text='Stock Parameters')
fig.for_each_trace(lambda t:  t.update(name = next(names)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

fig.show()

#### YEAR 2017

In [ ]:
#How does this year look like as a whole?
maindf['Date'] = pd.to_datetime(maindf['Date'], format='%Y-%m-%d')
year_2017 = maindf.loc[(maindf['Date'] >= '2017-01-01') & (maindf['Date'] <= '2017-12-31')]
year_2017.drop(year_2017[['Adj Close','Volume']],axis=1)

In [ ]:
#how is the price distributed on the basis of month?
monthly= year_2017.groupby(year_2017['Date'].dt.strftime('%B'))[['Open','Close']].mean()
month_name = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August',
             'September', 'October', 'November', 'December']
monthly = monthly.reindex(month_name, axis=0)
monthly

In [ ]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x=monthly .index,
    y=monthly ['Open'],
    name='Stock Open Price'
))
fig.add_trace(go.Bar(
    x=monthly .index,
    y=monthly ['Close'],
    name='Stock Close Price',
))

fig.update_layout(barmode='group', xaxis_tickangle=-45,
                  title='2017 Monthly comparative analysis')
fig.show()

**What about the opening and closing prices for months in the year**

In [ ]:
year_2017.groupby(year_2017['Date'].dt.strftime('%B'))['Low'].min()
monthvise_high = year_2017.groupby(maindf['Date'].dt.strftime('%B'))['High'].max()
monthvise_high = monthvise_high.reindex(month_name, axis=0)

monthvise_low = year_2017.groupby(year_2017['Date'].dt.strftime('%B'))['Low'].min()
monthvise_low = monthvise_low.reindex(month_name, axis=0)

fig = go.Figure()
fig.add_trace(go.Bar(
    x=monthvise_high.index,
    y=monthvise_high,
    name='Stock high Price',
))
fig.add_trace(go.Bar(
    x=monthvise_low.index,
    y=monthvise_low,
    name='Stock low Price',
))

fig.update_layout(barmode='group',
                  title='High and Low Price 2017 (Monthly)')
fig.show()

**Now lets get a more combined signature**

In [ ]:
names = cycle(['Stock Open Price','Stock Close Price','Stock High Price','Stock Low Price'])

fig = px.line(year_2017, x=year_2017.Date, y=[year_2017['Open'], year_2017['Close'],
                                          year_2017['High'], year_2017['Low']],
             labels={'Date': 'Date','value':'Stock value'})
fig.update_layout(title_text='Stock analysis chart 2017', font_size=15, font_color='black',legend_title_text='Stock Parameters')
fig.for_each_trace(lambda t:  t.update(name = next(names)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

fig.show()

#### YEAR 2018

In [ ]:
#How does this year look like as a whole?
maindf['Date'] = pd.to_datetime(maindf['Date'], format='%Y-%m-%d')
year_2018 = maindf.loc[(maindf['Date'] >= '2018-01-01') & (maindf['Date'] <= '2018-12-31')]
year_2018.drop(year_2018[['Adj Close','Volume']],axis=1)

In [ ]:
#how is the price distributed on the basis of month?
monthly= year_2018.groupby(year_2018['Date'].dt.strftime('%B'))[['Open','Close']].mean()
month_name = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August',
             'September', 'October', 'November', 'December']
monthly = monthly.reindex(month_name, axis=0)
monthly

In [ ]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x=monthly .index,
    y=monthly ['Open'],
    name='Stock Open Price'
))
fig.add_trace(go.Bar(
    x=monthly .index,
    y=monthly ['Close'],
    name='Stock Close Price',
))

fig.update_layout(barmode='group', xaxis_tickangle=-45,
                  title='2018 Monthly comparative analysis')
fig.show()

**What about the opening and closing prices for months in the year**

In [ ]:
year_2018.groupby(year_2018['Date'].dt.strftime('%B'))['Low'].min()
monthvise_high = year_2018.groupby(maindf['Date'].dt.strftime('%B'))['High'].max()
monthvise_high = monthvise_high.reindex(month_name, axis=0)

monthvise_low = year_2018.groupby(year_2018['Date'].dt.strftime('%B'))['Low'].min()
monthvise_low = monthvise_low.reindex(month_name, axis=0)

fig = go.Figure()
fig.add_trace(go.Bar(
    x=monthvise_high.index,
    y=monthvise_high,
    name='Stock high Price',
))
fig.add_trace(go.Bar(
    x=monthvise_low.index,
    y=monthvise_low,
    name='Stock low Price',
))

fig.update_layout(barmode='group',
                  title='High and Low Price 2018 (Monthly)')
fig.show()

**Now lets get a more combined signature**

In [ ]:
names = cycle(['Stock Open Price','Stock Close Price','Stock High Price','Stock Low Price'])

fig = px.line(year_2018, x=year_2018.Date, y=[year_2018['Open'], year_2018['Close'],
                                          year_2018['High'], year_2018['Low']],
             labels={'Date': 'Date','value':'Stock value'})
fig.update_layout(title_text='Stock analysis chart', font_size=15, font_color='black',legend_title_text='Stock Parameters')
fig.for_each_trace(lambda t:  t.update(name = next(names)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

fig.show()

In [ ]:
#How does this year look like as a whole?
maindf['Date'] = pd.to_datetime(maindf['Date'], format='%Y-%m-%d')
year_2019 = maindf.loc[(maindf['Date'] >= '2019-01-01') & (maindf['Date'] <= '2019-12-31')]
year_2019.drop(year_2019[['Adj Close','Volume']],axis=1)

In [ ]:
#how is the price distributed on the basis of month?
monthly= year_2019.groupby(year_2019['Date'].dt.strftime('%B'))[['Open','Close']].mean()
month_name = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August',
             'September', 'October', 'November', 'December']
monthly = monthly.reindex(month_name, axis=0)
monthly

In [ ]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x=monthly .index,
    y=monthly ['Open'],
    name='Stock Open Price'
))
fig.add_trace(go.Bar(
    x=monthly .index,
    y=monthly ['Close'],
    name='Stock Close Price',
))

fig.update_layout(barmode='group', xaxis_tickangle=-45,
                  title='2019 Monthly comparative analysis')
fig.show()

**What about the opening and closing prices for months in the year**

In [ ]:
year_2019.groupby(year_2019['Date'].dt.strftime('%B'))['Low'].min()
monthvise_high = year_2019.groupby(maindf['Date'].dt.strftime('%B'))['High'].max()
monthvise_high = monthvise_high.reindex(month_name, axis=0)

monthvise_low = year_2019.groupby(year_2019['Date'].dt.strftime('%B'))['Low'].min()
monthvise_low = monthvise_low.reindex(month_name, axis=0)

fig = go.Figure()
fig.add_trace(go.Bar(
    x=monthvise_high.index,
    y=monthvise_high,
    name='Stock high Price',
))
fig.add_trace(go.Bar(
    x=monthvise_low.index,
    y=monthvise_low,
    name='Stock low Price',
))

fig.update_layout(barmode='group',
                  title='High and Low Price 2019 (Monthly)')
fig.show()

**Now lets get a more combined signature**

In [ ]:
names = cycle(['Stock Open Price','Stock Close Price','Stock High Price','Stock Low Price'])

fig = px.line(year_2019, x=year_2019.Date, y=[year_2019['Open'], year_2019['Close'],
                                          year_2019['High'], year_2019['Low']],
             labels={'Date': 'Date','value':'Stock value'})
fig.update_layout(title_text='Stock analysis chart', font_size=15, font_color='black',legend_title_text='Stock Parameters')
fig.for_each_trace(lambda t:  t.update(name = next(names)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

fig.show()

#### YEAR 2015

In [ ]:
#How does this year look like as a whole?
maindf['Date'] = pd.to_datetime(maindf['Date'], format='%Y-%m-%d')
year_2020 = maindf.loc[(maindf['Date'] >= '2020-01-01') & (maindf['Date'] <= '2020-12-31')]
year_2020.drop(year_2020[['Adj Close','Volume']],axis=1)

In [ ]:
#how is the price distributed on the basis of month?
monthly= year_2020.groupby(year_2020['Date'].dt.strftime('%B'))[['Open','Close']].mean()
month_name = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August',
             'September', 'October', 'November', 'December']
monthly = monthly.reindex(month_name, axis=0)
monthly

In [ ]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x=monthly .index,
    y=monthly ['Open'],
    name='Stock Open Price'
))
fig.add_trace(go.Bar(
    x=monthly .index,
    y=monthly ['Close'],
    name='Stock Close Price',
))

fig.update_layout(barmode='group', xaxis_tickangle=-45,
                  title='2020 Monthly comparative analysis')
fig.show()

**What about the opening and closing prices for months in the year**

In [ ]:
year_2020.groupby(year_2020['Date'].dt.strftime('%B'))['Low'].min()
monthvise_high = year_2020.groupby(maindf['Date'].dt.strftime('%B'))['High'].max()
monthvise_high = monthvise_high.reindex(month_name, axis=0)

monthvise_low = year_2020.groupby(year_2020['Date'].dt.strftime('%B'))['Low'].min()
monthvise_low = monthvise_low.reindex(month_name, axis=0)

fig = go.Figure()
fig.add_trace(go.Bar(
    x=monthvise_high.index,
    y=monthvise_high,
    name='Stock high Price',
))
fig.add_trace(go.Bar(
    x=monthvise_low.index,
    y=monthvise_low,
    name='Stock low Price',
))

fig.update_layout(barmode='group',
                  title='High and Low Price 2020 (Monthly)')
fig.show()

**Now lets get a more combined signature**

In [ ]:
names = cycle(['Stock Open Price','Stock Close Price','Stock High Price','Stock Low Price'])

fig = px.line(year_2020, x=year_2020.Date, y=[year_2020['Open'], year_2020['Close'],
                                          year_2020['High'], year_2020['Low']],
             labels={'Date': 'Date','value':'Stock value'})
fig.update_layout(title_text='Stock analysis chart', font_size=15, font_color='black',legend_title_text='Stock Parameters')
fig.for_each_trace(lambda t:  t.update(name = next(names)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

fig.show()

#### YEAR 2021

In [ ]:
#How does this year look like as a whole?
maindf['Date'] = pd.to_datetime(maindf['Date'], format='%Y-%m-%d')
year_2021 = maindf.loc[(maindf['Date'] >= '2021-01-01') & (maindf['Date'] <= '2021-12-31')]
year_2021.drop(year_2021[['Adj Close','Volume']],axis=1)

In [ ]:
#how is the price distributed on the basis of month?
monthly= year_2021.groupby(year_2021['Date'].dt.strftime('%B'))[['Open','Close']].mean()
month_name = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August',
             'September', 'October', 'November', 'December']
monthly = monthly.reindex(month_name, axis=0)
monthly

In [ ]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x=monthly .index,
    y=monthly ['Open'],
    name='Stock Open Price'
))
fig.add_trace(go.Bar(
    x=monthly .index,
    y=monthly ['Close'],
    name='Stock Close Price',
))

fig.update_layout(barmode='group', xaxis_tickangle=-45,
                  title='2021 Monthly comparative analysis')
fig.show()

**What about the opening and closing prices for months in the year**

In [ ]:
year_2021.groupby(year_2021['Date'].dt.strftime('%B'))['Low'].min()
monthvise_high = year_2021.groupby(maindf['Date'].dt.strftime('%B'))['High'].max()
monthvise_high = monthvise_high.reindex(month_name, axis=0)

monthvise_low = year_2021.groupby(year_2021['Date'].dt.strftime('%B'))['Low'].min()
monthvise_low = monthvise_low.reindex(month_name, axis=0)

fig = go.Figure()
fig.add_trace(go.Bar(
    x=monthvise_high.index,
    y=monthvise_high,
    name='Stock high Price',
))
fig.add_trace(go.Bar(
    x=monthvise_low.index,
    y=monthvise_low,
    name='Stock low Price',
))

fig.update_layout(barmode='group',
                  title='High and Low Price 2021 (Monthly)')
fig.show()

**Now lets get a more combined signature**

In [ ]:
names = cycle(['Stock Open Price','Stock Close Price','Stock High Price','Stock Low Price'])

fig = px.line(year_2021, x=year_2021.Date, y=[year_2021['Open'], year_2021['Close'],
                                          year_2021['High'], year_2021['Low']],
             labels={'Date': 'Date','value':'Stock value'})
fig.update_layout(title_text='Stock analysis chart', font_size=15, font_color='black',legend_title_text='Stock Parameters')
fig.for_each_trace(lambda t:  t.update(name = next(names)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

fig.show()

#### YEAR 2022

In [ ]:
#How does this year look like as a whole?
maindf['Date'] = pd.to_datetime(maindf['Date'], format='%Y-%m-%d')
year_2022 = maindf.loc[(maindf['Date'] >= '2022-01-01') & (maindf['Date'] <= '2022-12-31')]
year_2022.drop(year_2022[['Adj Close','Volume']],axis=1)

In [ ]:
#how is the price distributed on the basis of month?
monthly= year_2022.groupby(year_2022['Date'].dt.strftime('%B'))[['Open','Close']].mean()
month_name = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August',
             'September', 'October', 'November', 'December']
monthly = monthly.reindex(month_name, axis=0)
monthly

In [ ]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x=monthly .index,
    y=monthly ['Open'],
    name='Stock Open Price'
))
fig.add_trace(go.Bar(
    x=monthly .index,
    y=monthly ['Close'],
    name='Stock Close Price',
))

fig.update_layout(barmode='group', xaxis_tickangle=-45,
                  title='2022 Monthly comparative analysis')
fig.show()

**What about the opening and closing prices for months in the year**

In [ ]:
year_2022.groupby(year_2022['Date'].dt.strftime('%B'))['Low'].min()
monthvise_high = year_2022.groupby(maindf['Date'].dt.strftime('%B'))['High'].max()
monthvise_high = monthvise_high.reindex(month_name, axis=0)

monthvise_low = year_2022.groupby(year_2022['Date'].dt.strftime('%B'))['Low'].min()
monthvise_low = monthvise_low.reindex(month_name, axis=0)

fig = go.Figure()
fig.add_trace(go.Bar(
    x=monthvise_high.index,
    y=monthvise_high,
    name='Stock high Price',
))
fig.add_trace(go.Bar(
    x=monthvise_low.index,
    y=monthvise_low,
    name='Stock low Price',
))

fig.update_layout(barmode='group',
                  title='High and Low Price 2022 (Monthly)')
fig.show()

**Now lets get a more combined signature**

In [ ]:
names = cycle(['Stock Open Price','Stock Close Price','Stock High Price','Stock Low Price'])

fig = px.line(year_2022, x=year_2022.Date, y=[year_2022['Open'], year_2022['Close'],
                                          year_2022['High'], year_2022['Low']],
             labels={'Date': 'Date','value':'Stock value'})
fig.update_layout(title_text='Stock analysis chart', font_size=15, font_color='black',legend_title_text='Stock Parameters')
fig.for_each_trace(lambda t:  t.update(name = next(names)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

fig.show()

# Overall Analysis from 2014-2022

In [ ]:
maindf['Date'] = pd.to_datetime(maindf['Date'], format='%Y-%m-%d')

y_overall = maindf.loc[(maindf['Date'] >= '2014-09-17')
                     & (maindf['Date'] <= '2022-02-19')]

y_overall.drop(y_overall[['Adj Close','Volume']],axis=1)

In [ ]:
monthvise= y_overall.groupby(y_overall['Date'].dt.strftime('%B'))[['Open','Close']].mean()
new_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August',
             'September', 'October', 'November', 'December']
monthvise = monthvise.reindex(new_order, axis=0)
monthvise

In [ ]:
names = cycle(['Stock Open Price','Stock Close Price','Stock High Price','Stock Low Price'])

fig = px.line(y_overall, x=y_overall.Date, y=[y_overall['Open'], y_overall['Close'],
                                          y_overall['High'], y_overall['Low']],
             labels={'Date': 'Date','value':'Stock value'})
fig.update_layout(title_text='Stock analysis chart', font_size=15, font_color='black',legend_title_text='Stock Parameters')
fig.for_each_trace(lambda t:  t.update(name = next(names)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

fig.show()



*   List item
*   List item



# Model Building

*However before we start developing our forecasting model, we could note that bitcoin price has fluctuated considerably between 2014 and 2022. Rising up to 15000 dollars and falling as far as 3000 dollars. Such massive fluctuation would negatively affect our model. To this end, it would be wiser to build our model using one year time-series data rathar than wholesome data.*

But first, lets take all closing data

In [ ]:
closedf = maindf[['Date','Close']]
print("Shape of close dataframe:", closedf.shape)

In [ ]:
fig = px.line(closedf, x=closedf.Date, y=closedf.Close,labels={'date':'Date','close':'Close Stock'})
fig.update_traces(marker_line_width=2, opacity=0.8, marker_line_color='orange')
fig.update_layout(title_text='Whole period of timeframe of Bitcoin close price 2014-2022', plot_bgcolor='white',
                  font_size=15, font_color='black')
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Okay, Based on our argument, we would slice one year for our model. Lets work with 2021

In [ ]:
closedf = closedf[closedf['Date'] > '2021-02-19']
close_stock = closedf.copy()
print("Total data for prediction: ",closedf.shape[0])

In [ ]:
closedf

In [ ]:
fig = px.line(closedf, x=closedf.Date, y=closedf.Close,labels={'date':'Date','close':'Close Stock'})
fig.update_traces(marker_line_width=2, opacity=0.8, marker_line_color='orange')
fig.update_layout(title_text='Considered period to predict Bitcoin close price',
                  plot_bgcolor='white', font_size=15, font_color='black')
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

We now normalize our data....

In [ ]:
# This line of code implements normalization using minmaxscaler technique
del closedf['Date']
scaler=MinMaxScaler(feature_range=(0,1))
closedf=scaler.fit_transform(np.array(closedf).reshape(-1,1))
print(closedf.shape)

*Now lets divide our dataset into training and testing to mitigate overfitting...*

In [ ]:
# we keep the training set as 60% and 40% testing set
training_size=int(len(closedf)*0.60)
test_size=len(closedf)-training_size
train_data,test_data=closedf[0:training_size,:],closedf[training_size:len(closedf),:1]
print("train_data: ", train_data.shape)
print("test_data: ", test_data.shape)

We will need to transform our dataset to time-series format. For this analysis, we will use a time step of 20

In [ ]:
# convert an array of values into a dataset matrix

def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return np.array(dataX), np.array(dataY)

In [ ]:
time_step = 20
X_train, y_train = create_dataset(train_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)
X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)

print("X_train: ", X_train.shape)
print("y_train: ", y_train.shape)
print("X_test: ", X_test.shape)
print("y_test", y_test.shape)

We will build using the LSTM model.

#### LSTM MODEL

An LSTM (Long Short-Term Memory) model is a type of recurrent neural network (RNN) that is designed to handle long-term dependencies in sequential data. It is widely used in various applications such as natural language processing, speech recognition, and time series prediction.

![](https://dezyre.gumlet.io/images/blog/lstm-model/The_Architecture_of_the_LSTM_Model.png?w=900&dpr=1.0)

Unlike traditional RNNs, LSTM models incorporate memory cells that capture and store information over time. This allows them to retain important information from the past and make more accurate predictions based on long-term dependencies. The memory cells are controlled by gates that regulate the flow of information, making the LSTM model capable of selectively remembering or forgetting information.

In [ ]:
model=Sequential()
model.add(LSTM(10,input_shape=(None,1),activation="relu"))
model.add(Dense(1))
model.compile(loss="mean_squared_error",optimizer="adam")
history = model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=200,batch_size=32,verbose=1)

Lets have a look at some loss and validation plots

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(loss))

plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend(loc=0)
plt.figure()


plt.show()

*That looks good. Now, time for some predictions...*

In [ ]:
train_predict=model.predict(X_train)
test_predict=model.predict(X_test)
train_predict.shape, test_predict.shape

## Evaluation of Model Performance

In [ ]:
# Reverse transformation
train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)
original_ytrain = scaler.inverse_transform(y_train.reshape(-1,1))
original_ytest = scaler.inverse_transform(y_test.reshape(-1,1))

#Computatoin of errors# Evaluation metrices RMSE and MAE
print("Train data RMSE: ", math.sqrt(mean_squared_error(original_ytrain,train_predict)))
print("Train data MSE: ", mean_squared_error(original_ytrain,train_predict))
print("Train data MAE: ", mean_absolute_error(original_ytrain,train_predict))
print("-------------------------------------------------------------------------------------")
print("Test data RMSE: ", math.sqrt(mean_squared_error(original_ytest,test_predict)))
print("Test data MSE: ", mean_squared_error(original_ytest,test_predict))
print("Test data MAE: ", mean_absolute_error(original_ytest,test_predict))

print("Train data R2 score:", r2_score(original_ytrain, train_predict))
print("Test data R2 score:", r2_score(original_ytest, test_predict))

print("Train data explained variance regression score:",
      explained_variance_score(original_ytrain, train_predict))
print("Test data explained variance regression score:",
      explained_variance_score(original_ytest, test_predict))



# Comparision of original stock close price and predicted close price

In [ ]:
# shift train predictions for plotting

look_back=time_step
trainPredictPlot = np.empty_like(closedf)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict
print("Train predicted data: ", trainPredictPlot.shape)

# shift test predictions for plotting
testPredictPlot = np.empty_like(closedf)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(train_predict)+(look_back*2)+1:len(closedf)-1, :] = test_predict
print("Test predicted data: ", testPredictPlot.shape)

names = cycle(['Original close price','Train predicted close price','Test predicted close price'])


plotdf = pd.DataFrame({'date': close_stock['Date'],
                       'original_close': close_stock['Close'],
                      'train_predicted_close': trainPredictPlot.reshape(1,-1)[0].tolist(),
                      'test_predicted_close': testPredictPlot.reshape(1,-1)[0].tolist()})

fig = px.line(plotdf,x=plotdf['date'], y=[plotdf['original_close'],plotdf['train_predicted_close'],
                                          plotdf['test_predicted_close']],
              labels={'value':'Stock price','date': 'Date'})
fig.update_layout(title_text='Comparision between original close price vs predicted close price',
                  plot_bgcolor='white', font_size=15, font_color='black', legend_title_text='Close Price')
fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

***Okay, why dont we try to make predictions for the next 30 days***

In [ ]:
x_input=test_data[len(test_data)-time_step:].reshape(1,-1)
temp_input=list(x_input)
temp_input=temp_input[0].tolist()

from numpy import array

lst_output=[]
n_steps=time_step
i=0
pred_days = 30
while(i<pred_days):

    if(len(temp_input)>time_step):

        x_input=np.array(temp_input[1:])
        #print("{} day input {}".format(i,x_input))
        x_input = x_input.reshape(1,-1)
        x_input = x_input.reshape((1, n_steps, 1))

        yhat = model.predict(x_input, verbose=0)
        #print("{} day output {}".format(i,yhat))
        temp_input.extend(yhat[0].tolist())
        temp_input=temp_input[1:]
        #print(temp_input)

        lst_output.extend(yhat.tolist())
        i=i+1

    else:

        x_input = x_input.reshape((1, n_steps,1))
        yhat = model.predict(x_input, verbose=0)
        temp_input.extend(yhat[0].tolist())

        lst_output.extend(yhat.tolist())
        i=i+1

print("Output of predicted next days: ", len(lst_output))

***QUite a lot of numbers. May make more sense pictorially***

In [ ]:
last_days=np.arange(1,time_step+1)
day_pred=np.arange(time_step+1,time_step+pred_days+1)
print(last_days)
print(day_pred)

In [ ]:
temp_mat = np.empty((len(last_days)+pred_days+1,1))
temp_mat[:] = np.nan
temp_mat = temp_mat.reshape(1,-1).tolist()[0]

last_original_days_value = temp_mat
next_predicted_days_value = temp_mat

last_original_days_value[0:time_step+1] = scaler.inverse_transform(closedf[len(closedf)-time_step:]).reshape(1,-1).tolist()[0]
next_predicted_days_value[time_step+1:] = scaler.inverse_transform(np.array(lst_output).reshape(-1,1)).reshape(1,-1).tolist()[0]

new_pred_plot = pd.DataFrame({
    'last_original_days_value':last_original_days_value,
    'next_predicted_days_value':next_predicted_days_value
})

names = cycle(['Last 15 days close price','Predicted next 30 days close price'])

fig = px.line(new_pred_plot,x=new_pred_plot.index, y=[new_pred_plot['last_original_days_value'],
                                                      new_pred_plot['next_predicted_days_value']],
              labels={'value': 'Stock price','index': 'Timestamp'})
fig.update_layout(title_text='Compare last 15 days vs next 30 days',
                  plot_bgcolor='white', font_size=15, font_color='black',legend_title_text='Close Price')

fig.for_each_trace(lambda t:  t.update(name = next(names)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()



##### Great. How does our prediction look like with the closing stock price and predictions joined together?

In [ ]:
lstmdf=closedf.tolist()
lstmdf.extend((np.array(lst_output).reshape(-1,1)).tolist())
lstmdf=scaler.inverse_transform(lstmdf).reshape(1,-1).tolist()[0]

names = cycle(['Close price'])

fig = px.line(lstmdf,labels={'value': 'Stock price','index': 'Timestamp'})
fig.update_layout(title_text='Plotting whole closing stock price with prediction',
                  plot_bgcolor='white', font_size=15, font_color='black',legend_title_text='Stock')

fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

#### Well, It looks like we have a fairly good model.
##### N/B Real life crypto-currency signatures are more volatile. So, employ model with caution!!!

![](https://as2.ftcdn.net/v2/jpg/00/95/28/15/1000_F_95281528_1f0kYMFUIZZ3bgLGIxkdKuLet2M5us0l.jpg)